In [2]:
%matplotlib ipympl
# %matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
from utils import create_session_config
import tensorflow_probability as tp
import numpy as np
from models.utils import covariance,correlation_matrix,tf_cov,assert_cov_positive_definite,corr,print_tensor,print_tensors,constrain_cdf
from models.nn_pdf_common import transform_x,density_estimator,create_pdf_layer_mv,create_partially_monotone_dense_layer,create_monotone_dense_layer,create_positive_weights
from matplotlib import pyplot as plt
from scipy.stats import multivariate_normal
from flags import FLAGS
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
import os
np.set_printoptions(threshold=np.inf)
from data.registry import uci_parkinsons_joint
from models.utils import get_train_inputs,get_eval_inputs
from models.rnade_deep import rnade_deep_model
from experiment.exporters import BestResultExporter

/home/pawel/tensorflow/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/pawel/tensorflow/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/pawel/tensorflow/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/pawel/tensorflow/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/pawel/tensorflow/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/pawel/tensorflow/lib/python3.5/importli

In [5]:
FLAGS.data_set="uci_parkinsons_joint"
data_loader = uci_parkinsons_joint() 

# tf.reset_default_graph()
# tf.set_random_seed(10)

# ds = get_train_inputs(data_loader, batch_size=200)
# iter = tf.data.Iterator.from_structure(ds.output_types,ds.output_shapes)
# training_init_op = iter.make_initializer(ds)
# batch=iter.get_next()

loaded data: uci_parkinsons_joint.npz


In [10]:
tf.reset_default_graph()
params = {}
params['arch']=[10,10,10]
params['batch_size'] = 200
params['k_mix']=3
params['num_ensambles']=2
params['learning_rate']=0.001
params['x_size'] = data_loader.train_x.shape[1]
params['y_size'] = data_loader.train_y.shape[1]
params['components_distribution'] = 'normal'

session_config=create_session_config()
run_config = tf.estimator.RunConfig(model_dir='/home/pawel/PycharmProjects/RM_labs/tmp4',
                                            save_summary_steps=FLAGS.save_summary_steps,
                                            save_checkpoints_steps=100,
                                            save_checkpoints_secs=None,
                                            log_step_count_steps=1000, keep_checkpoint_max=3,
                                            session_config=session_config,
                                            tf_random_seed=10)

estimator = tf.estimator.Estimator(config=run_config, model_fn=rnade_deep_model, params=params)


early_stopping_hook = tf.contrib.estimator.stop_if_no_decrease_hook(
                estimator,
                metric_name='loss',
                max_steps_without_decrease=FLAGS.max_steps_without_decrease,
                run_every_steps=100,
                run_every_secs=None)

hooks = [early_stopping_hook]
exporter = BestResultExporter()
eval_spec = tf.estimator.EvalSpec(
    input_fn=lambda: get_eval_inputs(data_loader, batch_size=params['batch_size']), steps=None, start_delay_secs=0,
    throttle_secs=FLAGS.eval_throttle_secs, hooks=hooks, exporters=exporter)
    
train_spec = tf.estimator.TrainSpec(input_fn=lambda: get_train_inputs(data_loader, batch_size=params['batch_size']), max_steps=None,
                                                hooks=hooks)
tf.estimator.train_and_evaluate(estimator, train_spec, eval_spec)

# with tf.Session(config=tf.ConfigProto(gpu_options=tf.GPUOptions(per_process_gpu_memory_fraction=0.5))) as session:
#     session.run(tf.global_variables_initializer())
#     train_loop(30000, train_op)


INFO:tensorflow:Using config: {'_model_dir': '/home/pawel/PycharmProjects/RM_labs/tmp4', '_keep_checkpoint_every_n_hours': 10000, '_evaluation_master': '', '_tf_random_seed': 10, '_task_id': 0, '_save_checkpoints_secs': None, '_session_config': gpu_options {
  per_process_gpu_memory_fraction: 0.035
}
, '_keep_checkpoint_max': 3, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f5071a26668>, '_global_id_in_cluster': 0, '_save_checkpoints_steps': 100, '_master': '', '_num_ps_replicas': 0, '_save_summary_steps': 500, '_task_type': 'worker', '_protocol': None, '_eval_distribute': None, '_service': None, '_device_fn': None, '_log_step_count_steps': 1000, '_is_chief': True, '_train_distribute': None, '_num_worker_replicas': 1}
INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after every checkpoint. Checkpoint frequency is determined based on RunConfig arguments: s

NotFoundError: Restoring from checkpoint failed. This is most likely due to a Variable name or other graph key that is missing from the checkpoint. Please ensure that you have not altered the graph expected based on the checkpoint. Original error:

Key dense_10/bias not found in checkpoint
	 [[{{node save/RestoreV2}} = RestoreV2[dtypes=[DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_FLOAT, ..., DT_FLOAT, DT_FLOAT, DT_FLOAT, DT_INT64, DT_BOOL], _device="/job:localhost/replica:0/task:0/device:CPU:0"](_arg_save/Const_0_0, save/RestoreV2/tensor_names, save/RestoreV2/shape_and_slices)]]
	 [[{{node save/RestoreV2/_301}} = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device_incarnation=1, tensor_name="edge_306_save/RestoreV2", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:GPU:0"]()]]

Caused by op 'save/RestoreV2', defined at:
  File "/usr/lib/python3.5/runpy.py",

In [3]:
input=tf.constant([[1,2,3,4,5,6,7],[8,9,10,11,12,13,14]],dtype=tf.float32)
output=tf.constant([[1,2,3,4,5,6,7],[8,9,10,11,12,13,14]],dtype=tf.float32)
D=7
idx_sampler = tp.distributions.Categorical(probs=np.ones(D)/D, allow_nan_stats=False)
idx = idx_sampler.sample()
ordering = tf.random_shuffle(tf.range(0, D))
mask_idx = tf.slice(ordering, [idx], [-1])
mask = tf.where(tf.reduce_any(tf.equal(tf.reshape(tf.range(0, D),[-1,1]),mask_idx), axis=1), x=tf.zeros(D, dtype=tf.float32), y=tf.ones(D, dtype=tf.float32))
mask_matrix=tf.diag(mask)
input = tf.concat([tf.tile(tf.expand_dims(mask,0),[input.shape[0].value,1]),tf.matmul(input, mask_matrix)], axis=1) 

with tf.Session(config=tf.ConfigProto(gpu_options=tf.GPUOptions(per_process_gpu_memory_fraction=0.5))) as session:
    session.run(tf.global_variables_initializer())
    idx_,ordering_,mask_,input_,output_,mask_matrix_ = session.run([idx,ordering,mask,input,output,mask_matrix])
    
    print("index: ",idx_)
    print("ordering: ",ordering_)
    
    print("masked input\n", input_)
    print("masked output\n", output_)

index:  6
ordering:  [6 1 5 0 4 2 3]
masked input
 [[ 1.  1.  1.  0.  1.  1.  1.  1.  2.  3.  0.  5.  6.  7.]
 [ 1.  1.  1.  0.  1.  1.  1.  8.  9. 10.  0. 12. 13. 14.]]
masked output
 [[ 1.  2.  3.  4.  5.  6.  7.]
 [ 8.  9. 10. 11. 12. 13. 14.]]
